In [21]:
#import libraries
import os
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta
from dotenv import load_dotenv
import hvplot.pandas
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer


import os
import numpy as np
import pandas as pd
import random

In [22]:
#pull in data from previously pulled news articles sentiments 
data = Path('news_sentiment.csv')
news_sentiment_df = pd.read_csv(data, index_col=0)
news_sentiment_df.head()

,vote_avg,protest_avg,election_avg
2021-01-11,-0.042335,-0.039450,0.021515
2021-01-10,-0.061795,-0.258725,-0.047390
2021-01-09,-0.178465,-0.143980,-0.182350
2021-01-08,-0.147315,-0.362070,-0.153045
2021-01-07,-0.338450,-0.361915,-0.238405


In [23]:
twitter_data = Path('TWTR.csv')
twitter_df = pd.read_csv(twitter_data, index_col=0)
twitter_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-10,46.700001,51.740002,46.540001,51.209999,51.209999,24147200
2020-12-11,50.619999,52.450001,49.970001,51.439999,51.439999,18020200
2020-12-14,51.970001,53.480000,51.799999,52.020000,52.020000,17418200
2020-12-15,52.020000,52.959999,51.610001,52.820000,52.820000,10868700
2020-12-16,54.770000,56.110001,53.639999,54.029999,54.029999,29099400


In [24]:
facebook_data = Path('FB.csv')
facebook_df = pd.read_csv(facebook_data, index_col=0)
facebook_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-12-10,201.660004,202.050003,200.149994,200.869995,200.869995,9475700
2019-12-11,200.279999,202.630005,200.279999,202.259995,202.259995,8036800
2019-12-12,202.350006,203.660004,194.100006,196.750000,196.750000,23734800
2019-12-13,196.399994,196.800003,193.169998,194.110001,194.110001,18798000
2019-12-16,195.270004,199.119995,194.809998,197.919998,197.919998,14911500


In [25]:
#Get percent returns on the closing price, drop other values 
twitter_df = twitter_df["Close"]
twitter_returns = twitter_df.pct_change().dropna()
twitter_returns = pd.DataFrame(twitter_returns)
twitter_returns.rename(columns = {"Close":"TWTR"}, inplace = True)
twitter_returns

,TWTR
Date,
2020-12-11,0.004491
2020-12-14,0.011275
2020-12-15,0.015379
2020-12-16,0.022908
2020-12-17,0.011105
2020-12-18,0.022698
2020-12-21,-0.022015
2020-12-22,0.004941
2020-12-23,-0.011109


In [26]:
facebook_df = facebook_df["Close"]
facebook_returns = facebook_df.pct_change().dropna()
facebook_returns = pd.DataFrame(facebook_returns)
facebook_returns.rename(columns = {"Close":"FB"}, inplace = True)
facebook_returns

,FB
Date,
2019-12-11,0.006920
2019-12-12,-0.027242
2019-12-13,-0.013418
2019-12-16,0.019628
2019-12-17,0.002375
...,...
2021-01-05,0.007548
2021-01-06,-0.028269
2021-01-07,0.020622


In [27]:
stock_sentiment = news_sentiment_df.join(twitter_returns, how='inner')
stock_sentiment = stock_sentiment.join(facebook_returns, how='inner')
stock_sentiment

,vote_avg,protest_avg,election_avg,TWTR,FB
2021-01-11,-0.042335,-0.039450,0.021515,-0.064103,-0.040102
2021-01-08,-0.147315,-0.362070,-0.153045,-0.016243,-0.004354
2021-01-07,-0.338450,-0.361915,-0.238405,-0.017461,0.020622
2021-01-06,0.058345,-0.271630,-0.054620,-0.011507,-0.028269
2021-01-05,-0.099680,-0.307570,-0.004395,-0.011920,0.007548
2021-01-04,0.076610,-0.164505,-0.064460,0.007017,-0.015449
2020-12-31,-0.136875,-0.311000,-0.077385,-0.003313,0.004745
2020-12-30,-0.036210,-0.151215,-0.067085,-0.000552,-0.017740
2020-12-29,0.281975,-0.111845,-0.002515,-0.001286,-0.000794
2020-12-28,-0.008520,-0.196625,0.017560,0.008523,0.035901


In [28]:
stock_sentiment.corr().style.background_gradient()

,vote_avg,protest_avg,election_avg,TWTR,FB
vote_avg,1.000000,0.547732,0.264692,0.389354,-0.123198
protest_avg,0.547732,1.000000,-0.128203,-0.127300,-0.380359
election_avg,0.264692,-0.128203,1.000000,-0.082787,-0.003378
TWTR,0.389354,-0.127300,-0.082787,1.000000,0.472059
FB,-0.123198,-0.380359,-0.003378,0.472059,1.000000
